In [ ]:
# Gyakran változtatott konstansok

# Mekkora kontextusra történjen az illesztés
CONTEXT_LENGTH = None

# A teljes adatbázis pozitív címkéjű adatainak hány százalékával történjen a futás
POZITIV_PERCENTAGE_TO_KEEP = 100

# A teljes adatbázis negatív címkéjű adatainak hány százalékával történjen a futás
NEGATIV_PERCENTAGE_TO_KEEP = 50

# A teljes adatbázis semleges címkéjű adatainak hány százalékával történjen a futás
SEMLEGES_PERCENTAGE_TO_KEEP = 12

# Stop word-ök használata
USE_STOPWORDS = True

# Stemmelés használata
USE_STEM = True

In [ ]:
# Importok

import os
import random
import re

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
from nltk import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
# Konstansok

# Random seed értéke
SEED = 42

# Könyvtárspecifikus random seed-ek értéke
SHUFFLE_RANDOM_STATE = SEED
TRAIN_RANDOM_STATE = SEED
RANDOM_SEED = SEED
NP_SEED = SEED
TF_SEED = SEED
PYTHON_HASH_SEED = SEED
SGD_RANDOM_STATE = SEED

# A program számára fontos fejlécek értéke az adatbázisban
TEXT = 'Sentence'
FILTER = 'Entity'
START_TOKEN = 'START'
TOKEN_LEN = 'LEN'
Y_HEADER = 'LABEL'

# Az adatbázisban található értékek megfeleltetése a programban
LABELS = {
    "NEG": 0,
    "SEM": 1,
    "POZ": 2
}

In [ ]:
# Random seed-ek állítása

os.environ['PYTHONHASHSEED']=str(PYTHON_HASH_SEED)
np.random.seed(NP_SEED)
random.seed(RANDOM_SEED)
tf.random.set_seed(TF_SEED)
tf.compat.v1.set_random_seed(TF_SEED)

In [ ]:
# Text processing

def extend_context(text, context, left_index, right_index, context_length):
    """
    Kontextus összeállításáért felelős metódus.

    Parameters
    ----------
    text: str | list(str)
        A teljes szöveg, amelyből a kontextust kinyerjük.
    context: str
        Kezdetben csak célszemélyt tartalmazza, később a teljes kontextust.
    left_index: int
        Kezdetben a célszemély kezdőindexe a szövegben.
    right_index: int
        Kezdetben a célszemély utolsó tokenjének indexe a szövegben.
    context_length: int
        Mekkora legyen a kontextusméret.

    Returns
    -------
    str
        Az összeállított kontextus.
    """

    if context_length is None:
        context_length = len(text)
    if type(text) != list:
        text = text.split()
    start_size = 0
    while start_size < context_length:
        if left_index > 0:
            left_index -= 1
            context.insert(0, text[left_index])
            if len(text[left_index]) > 1:
                start_size += 1
        if right_index < len(text) -1:
            right_index += 1
            context.append(text[right_index])
            if len(text[right_index]) > 1:
                start_size += 1
        if left_index <= 0 and right_index >= len(text) -1:
            break
    return ' '.join(context)


def contextualize(index, context_length=None):
    """
    Metódus, amely inicializálja a kontextus összeállítását.

    Parameters
    ----------
    index: int
        Annak a dokumentumnak az indexe, amelynek a kontextusát szeretnénk kinyerni.
    context_length: int, optional
        Kontextus mérete. None esetén a teljes dokumentum a kontextus (default is None).

    Returns
    -------
    str
        Az összeállított kontextus.
    """

    text = ' '.join([i for i in re.split(r'( - |(?![%.-])\W)|(-e[\n ])', dataset[TEXT].iloc[index]) if i])
    context_start_index = int(dataset[START_TOKEN].iloc[index] - 1)
    context_stop_index = int(context_start_index+dataset[TOKEN_LEN].iloc[index] - 1)

    context_name_tokens = \
        [t for t in text.split()[context_start_index:context_stop_index+1]]

    context_list = context_name_tokens
    left_index = context_start_index
    right_index = context_stop_index
    context = extend_context(text, context_list, left_index, right_index, context_length)
    return context

def delete_labeled_rows(dataset, label, percentage_to_remain = 10):
    """
    Metódus, amely a paraméterben kapott címkével ellátott adatoknak csak a megadott százalékát tartja meg
    minden más adat törlésre kerül. A törlésre kerülő sorok véletlenszerűen kerülnek kiválasztásra.

    Parameters
    ----------
    dataset: pandas.DataFrame
        Az adatbázisból készített DataFrame.
    label: int
        A vizsgált címke, amire a törlés végre lesz hajtva.
    percentage_to_remain: int, optional
        Megadható, hogy a DataFrame-ben a megadott címkének hány százaléka maradjon a DataFrameben (default is 10).

    Returns
    -------
    pandas.DataFrame
        Az a DataFrame, amely az eldobott sorokat már nem tartalmazza az adott címkéből.
    """

    label_mapped = dataset[Y_HEADER].map(LABELS)
    neutral_ids = dataset.index[label_mapped == label].tolist()
    neutral_id_drop = set(random.sample(range(len(neutral_ids)), int(len(neutral_ids) * percentage_to_remain / 100)))
    ids_to_delete = [x for i,x in enumerate(neutral_ids) if not i in neutral_id_drop]
    return dataset.drop(ids_to_delete)

In [ ]:
# Adatok előkészítése

dataset = pd.read_csv('db/train.csv', sep=';')
dataset = shuffle(dataset, random_state=SHUFFLE_RANDOM_STATE)
dataset.info()

dataset = delete_labeled_rows(dataset, LABELS['SEM'], percentage_to_remain=SEMLEGES_PERCENTAGE_TO_KEEP)
dataset = delete_labeled_rows(dataset, LABELS['NEG'], percentage_to_remain=NEGATIV_PERCENTAGE_TO_KEEP)
dataset = delete_labeled_rows(dataset, LABELS['POZ'], percentage_to_remain=POZITIV_PERCENTAGE_TO_KEEP)

X_list = []
print(len(dataset.index))
for i  in range(len(dataset.index)):
    X_list.append(contextualize(i))

X = np.asarray(X_list)
y = dataset[Y_HEADER].values


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=TRAIN_RANDOM_STATE)

In [ ]:
# Adatok vizualizálása

def plot_label_counts(y, title='y labels'):
    """
    Adatok eloszlásának ábrázolása.

    Parameters
    ----------
    y: numpy.ndarray
        Az adathalmaz címkéi, amelyre a vizualizálás történik.
    title: str
        Az elkészített ábra címe.
    """

    unique, counts = np.unique(y, return_counts=True)
    b = dict(zip(unique, counts))
    b_values = [b['NEG'], b['SEM'], b['POZ']]
    print(b.values())
    plt.barh(range(len(b)), b_values, align='center', color=['pink', 'lightblue', 'lightgreen'])
    y_values = ["Negatív", "Semleges", "Pozitív"]
    y_axis = np.arange(0, 3, 1)
    plt.yticks(y_axis, y_values)
    plt.title(title)
    plt.xlabel('Number of Samples in training Set')
    plt.ylabel('Label')
    ax = plt.gca()
    for i, v in enumerate(b_values):
        plt.text(ax.get_xlim()[1]/100, i, str(v), color='blue', fontweight='bold')
    plt.show()

plot_label_counts(y_train, 'Train eloszlas')
plot_label_counts(y_test, 'Test eloszlas')

In [ ]:
# Kiértékelés vizualizálása szövegesen és ábrán

def evaluate(predict, labels):
    """
    A modell kiértékelése. Kiíratja az osztályozási eredményeket, a pontosságot, illetve az igazságmátrixot.

    Parameters
    ----------
    predict: numpy.ndarray
        A modell predikcióit tartalmazza a teszt adathalmazra.
    y: numpy.ndarray
        A teszt adathalmaz címkéit tartalmazza.
    """

    print('Classification report:')
    print(classification_report(labels, predict))
    print('Accuracy:')
    print(accuracy_score(labels, predict))

    print('Confusion matrix:')
    df_cm = pd.DataFrame(confusion_matrix(labels, predict, labels=['POZ', 'SEM', 'NEG']),
                         index=[i for i in ['POZ', 'SEM', 'NEG']],
                         columns=[i for i in ['POZ', 'SEM', 'NEG']])
    plt.figure(figsize=(10,7))
    hm = sn.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
    hm.set(ylabel='True label', xlabel='Predicted label')
    plt.show()

In [ ]:
# Stop word-ök letöltése

nltk.download('stopwords')

In [ ]:
# Korpuszok létrehozása a dokumentumokból

all_stopwords = []

def create_corpus_by_dataset(sentences):
    """
    Korpusz létrehozása. Stop word-ök és stemmelés alkalmazása, amennyiben azok engedélyezve vannak.

    Parameters
    ----------
    sentences: list of numpy.ndarray
        Az adathalmazban található dokumentumokat tartalmazó lista, amelyen a stop word-ök alkalmazva lesznek.

    Returns
    -------
    list of str
        Az elkészített korpusz.
    """

    global all_stopwords
    corpus = []
    for sen in sentences:
        sentence = re.sub('[^a-zA-ZöÖüÜóÓőŐúÚáÁűŰéÉíÍ]', ' ', sen)
        sentence = sentence.lower().split()
        if USE_STOPWORDS:
            all_stopwords = stopwords.words('hungarian')
            whitelist = ["ne", "nem", "se", "sem"]
            sentence = [word for word in sen.split() if (word not in all_stopwords or word in whitelist)
                 and len(word) > 1]
        if USE_STEM:
            ps = SnowballStemmer('hungarian')
            sentence = [ps.stem(word) for word in sentence]
        sentence = ' '.join(sentence)
        corpus.append(sentence)

    return corpus

X_train_clean = create_corpus_by_dataset(X_train)
X_test_clean = create_corpus_by_dataset(X_test)

In [ ]:
# Bag of words modell létrehozása és illesztése

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.002, ngram_range=(1,3))
x_train_model = cv.fit_transform(X_train_clean)

In [ ]:
# SGD osztályozó létrehozása és illesztése

from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier(random_state=SGD_RANDOM_STATE, learning_rate='constant',
                           eta0=0.01)
classifier.fit(x_train_model, y_train)

In [ ]:
# Beállított modellek kiértékelése

x_dev_model = cv.transform(X_test_clean)

dev_predict = classifier.predict(x_dev_model)

evaluate(dev_predict, y_test)

In [ ]:
# Grid Search előkészítése és illesztése

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
           ('vect', cv),
           ('clf', classifier)
])

parameters_to_tune = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (0.001, 0.002, 0.005, 0.01),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2), (1,3), (1,4)),
    'clf__max_iter': (500, 1000, 2000, 5000),
    'clf__eta0': (0.005, 0.01, 0.02, 0.05)
}
clf=GridSearchCV(pipeline,parameters_to_tune, n_jobs=-1, verbose=1)
clf.fit(X_train_clean, y_train)

In [ ]:
# Optimális hiperparaméterekkel beállított modellek részleteinek kiírása

print(clf.best_estimator_)

In [ ]:
# Predikálás az optimális hiperparaméterekkel beállított modelleken

pred2 = clf.predict(X_test_clean)
evaluate(pred2, y_test)